# An introduction to CSP-style concurrency with _aiochan_

## Setting the stage

### What is concurrency

*Aiochan* is a python library for CSP-style concurrency in python. So the first questions to ask are: what is concurrency? Why do we need concurrency? Aren't we doing just fine writing non-concurrent python?

By far the most important reason for want of concurreny is that *concurrency enables your program to deal with multiple (potentially different) things all at once*. For example, suppose you are writing a webserver. Let's say that your code goes like this:

```
while True:
    parse request from client
    do something with the request (which takes an unspecified amount of time)
    return the result to the client
```

This is all very good but if this is the *outmost loop* of your server, then it is easy to see that at most one client can be served at any one instant. If a user is doing an operation that takes, say, ten minutes to complete, then it is not too far-fetched to assume that other users will not be too happy.

Yes, I hear you say, "I have been writing python webservers using non-concurrent codes not too different from above for a long time, and it is definitely *not* true that only one client is served at any one instant". Well, most likely you are using some web frameworks and it is the framework that controls the outmost loop. In other words, your framework is managing all your concurrency whilst presenting a non-concurrent façade to you.

There is one big caveat that we need to clarify. Above we have defined concurrency as the ability to "deal with multiple things at once". Let's say that you have all the things *you* need to do written on a to-do list. Concurrency only means that you do not need to take the first item off the list, do it *and wait for the result*, then start with the second item. Your first item might well be "watch the news at 9am, then watch the news at 9pm, and find out what new things have happened since 9am". In this case non-concurrent behaviour means sitting there doing nothing after watching the 9am news until 9pm. As long as you *switch context* and starting doing something after the morning news you are in the realm of concurrent behaviour. Note that *you need neither a group of cronies to whom you can delegate your news watching, nor the rather unusual ability to watch two programs at once and perfectly understanding both, to enable concurrent behaviour*. Having a group of cronies doing stuff for you is a form of *parallelism*. Concurrency *enables* parallelism (it is useless to have many cronies if you need to wait for any one of them to complete their work before assigning work to the next one), but parallelism is not *necessary* for concurrency. Parallelism usually (but not always) makes things go faster. Concurrency can also make things go faster *even without parallelism*. In the case of computers, you *do not* need to have multiple processors to benefit from concurrency (in the case of python, this point is actually quite acute--see later when we discuss the GIL).

Now back to our webserver programs. How do you write a concurrent outmost loop then? By analogy with the todo list example, you are probably thinking along something like:

```
while True:
    parse request from client OR get the next thing to do 
    if request, put it into the things-to-do list
    if you have a thing to do, try do it, but not until completion, only until something we have to "wait for"
    Oh God, this is a mess! What if we have 10 things to do, how do we choose? Or nothing new to do, how do we "wait"?
    ???
```

It is not impossible to complete a program like this, but you are right to feel like juggling ten balls at once -- it is difficult, the solution is brittle, and you probably don't want to do this everyday (unless you are a professional juggler).

Here is the place for concurrent libraries, frameworks, or language constructs: as long as you follow certain rules, they enable you to have the benefit of concurrency without the need of professional juggling training. *Aiochan* is such a library.

To recap:

* Concurrency enables you to deal with multiple things at once.
* Concurrency has the potential to decrease latency and increase throughput.
* Concurrency is not parallelism but enables it.
* Concurrency frameworks, libraries and language constructs enable *you* to take advantange of concurrency without writing complicated and brittle code.

### What is CSP (or Communicating Sequential Processes)?

We have said that *aiochan* is a CSP-style concurrency library. It turns out we can go a long way towards understanding how to use *aiochan* by just understanding what *Communicating Sequential Processes* mean. (The name CSP first appears in a paper by C.A.R. Hoare in 1978, and some ideas of CSP exist even before that. So CSP is actually quite an old idea.)

Let's begin with the last word: *processes*. It seems we have immediately hit upon opportunities for great confusion: python has many packages that have "process" on their names, your operating system has something called "processes" or maybe not, depending on what OS you are using, etc. And they are not what we want to talk about now. *Process* is, unfortunately, one of thoses words in computing that is over-used in many subtly different ways. Here we shall be intentially vague: for us, a *process* is just a group of code that executes fairly independently from the outside world, a group of code that you can mentally think of as a whole entity. The quintessential example is that of a function: a function as a process goes from taking in arguments from the caller, and ends when returning to the caller.

What is *sequential* process, then? If you read the word literally, it means that statements or expressions in the process are executed or evaluated in strict order, from top to bottom. In python this is fairly accurate, if special provision is made for control statements such as `while`, `for` and stuff. A better word might be *deterministic*: even in the presence of control statements, if you know all your variables, it is possible to predict precisely what the next statement or expression to be executed or evaluated is. An example is

In [1]:
x = 10
x += 10
x *= 2
x -= 7
print(x)

33


The above calculates `((10 + 10) * 2) - 7 = 33` and is sequential. If your programming language instead calculates `((10 * 2) + 10) - 7 = 3` then your programming environment has some serious issues. So sequential programs are good, it is what we as humans expect.

However, it is actually very easy to have non-sequential programs. Let's first refactor the above program:

In [4]:
x = 10

def f():
    global x
    print('we start with the value', x)
    x += 10
    x *= 2
    x -= 7
    return x

print(f())

we start with the value 10
33


So far so good. But imagine you have two instances of the `f` process executing concurrently, you might be having some troubles. It is actually possible to illustrate this with python code, but for simplicity we will just give an illustration:

```
x = 10

def f1():                                              | def f2():
    global x                                           |
                                                       |     global x                                     
    print('f1 start with the value', x) # x == 10      |
                                                       |     print('f2 start with the value', x) # x == 10
    x += 10 # x == 20                                  |
                                                       |     x += 10 # x == 30
    x *= 2  # x == 40                                  |
                                                       |     x *= 2  # x == 80
    x -= 7  # x == 33                                  |
    return x                                           |
                                                       |     x -= 7  # x == 26
                                                       |     return x

print('f1 ->', f1())
print('f2 ->', f2())
```

In the above snippet, for the two functions, the order of the execution of statements within the two functions are as listed: from top to down. Note that though within each function itself the sequence is the same as before, when the two functions are taken together statements are interleaved.

We will get the results:

```
f1 start with the value 10
f1 start with the value 10
f1 -> 33
f2 -> 26
```

Now if you are only in control of `f1` you will be very much baffled. Worse, as you can try for yourself, by tweaking the order further you can get other results. The interleaving makes the execution, and hence the result, non-deterministic. Such processes are *not* considered sequential and not allowed (or, not encouraged) in CSP-style programming.

As you probably can see, the fix is actually not that hard: *don't modify global variables*. Any modifications you do must be local to your process (or in our case, function). You should try and see for yourself that if you follow this advice, then whatever the order of interleaving of the two functions, the results are not affected: they are deterministic and are thus considered *sequential*.

In functional languages, it is sometimes enforced that you cannot make any modifications at all --- any computation you do just returns new values without stepping on the old values. However, in python, this is both unnecessary and unnatural. We only need to disallow operations that can interfere with other processes.

Now, you ask, which disturbed mind would write something like our `f`? Well, be assured that that people who wrote `f` habour no ill intensions. The reason that they reach for global variables is most often because that they want to do some form of input/output (i.e., IO, note that the concept of IO is much broader than file or network accesses): we need to get stuff into our function to compute, and we need to notify others who are also computing what the results of our computations are.

If you think about it, IO is the whole point of computation: *we*, at our keyboards (or touch screens, or whatever your newest VR/AR interaction devices), input something for the computer to compute, the the computer needs to return the results to *us*. So programs without IO is pretty useless. And using global variables in this case is rather convenient: we just take something (input) that is put inside predetermined boxes (global variables), and when we are done, just put them back. Others, by inspecting the boxes, will know what we have done. By the way, at the lowest level, this is how our current computer architecture dictates. A "pure" function that "returns" something without reference to a box location *is* an illusion. But unfortunately, as we have seen, this crude arrangement results in people stepping on each other and chaos if there are no rules.

In the older mainstream languages, the solution is that we put stickers on the boxes when we want to operate on them: "in use --- don't touch until I'm done!" --- locks, semaphores, etc. This solves the problem, but using locks and similar *concurrency primitives* turn out to be rather delicate and error-prone. A classical example is that you and your friend both want to operate on two boxes A and B. You go forward and put your sticker on A, meanwhile your friend has already put his sticker on B. Now both of you are stuck: unless one of you back off, none can go forward. Preventing such *deadlocks* means having a whole lot of disciplines and guidelines to follow --- more training to become professional jugglers!

Is there a way out of this? Is there a way to avoid arduous juggler training while still doing concurrency? Yes, and this is what the *Communicating* part of CSP says.

The basic idea is this: when doing computations that must involve read-write IO, we do away with boxes. Instead, we specify *meeting points*, or *points of rendezvous*, for which IO is done. For example, you and your friend both wants a book. Instead of putting the book in a box so that both of you can do whatever you want with it whenever you want (and risking the book to be stolen), you just take the book away and do your processing with it. After you are satisfied, you and your friend *meet together* and you *hand off* the book to your friend. Once your friend has the book, she can do anything she wants with it, while you can no longer do anything with it at all. There is no longer any stepping over. If you *really* want your book again, you must arange with your friend for a hand-off again.

Such an arrangment is at least familiar (it is how private properties work). Note that it is principally different from putting stickers on boxes: you just take the book and go off! And the amazing thing is, this arrangement actually solves the majority of concurrency problems. To each *sequential process*, things look as if we are in a non-concurrent program: the only concurrent parts are when we want to do IO, we arrange for rendezvous points. No stickers. No locks. Much less opportunities for deadlocks. And privacy! Your friend don't need to have a fairly accurate picture of what you are up to in order to cooperate with you --- your ability to honour rendezvous arrangements is (almost) the only thing she requires of you! You and your friend *communicate* over rendezvous and go about your lives independently otherwise.

The rest of this tutorial will go into much more details in how to go about setting up and honouring rendezvous appointments, which in the context of *aiochan*, is called a *channel*, or `Chan`. But first, some environment setups have to be done first, which we will deal with in the next section.

To recap, in the context of CSP (*Communicating Sequantial Processes*),

* Processes are group of codes that can be considered as an independent entity.
* Sequential processes are processes that operate in deterministic order producing deterministic results, without danger of stepping over each other.
* Communicating sequantial processes are sequantial processes that do their (necessary) IO by using rendezvous points only.
* CSP style concurrency enables natural program logic resembling non-concurrent codes while at the same time eliminating most dangers of deadlocks and stepping over.

## Basic operations

* The sad reality of concurrency in python
    * Not initially designed for concurrency
    * When you start python, you are not in a situation where you can execute concurrent instructions directly
    * Explicitly delaring concurrent environment: processes, threads, asyncio loop
    * How we can keep things as simple as possible now
    * examples: concurrent programs (but non-communicating)

* The basic IO device: channel
    * channel IS the IO device for CSP
    * get and put
    * fan-in and fan-out
    * passing channels around

* The paramount `select`
    * non-determinism or choice
    * example: stopping
    * `select` is non-trivial
        * Java was designed with threads in mind, but needs java.util.concurrent as a library
        * Clojure was designed with concurrency in mind, has all the java primitives and libraries to use, has its additional (primitive) refs (atoms, agents, event stm (software transactional memory)), but _still_ needs core.async as a library
        * You can say that the whole core.async is very much built to enable `select` (called `alt!`, `alts!`, `alt!!`, `alts!!` there)

* Understanding channel buffering
    * why buffer: even more concurrency, even more control
    * the basics: fixed buffer, dropping buffer, sliding buffer
    * channel and locks and semaphores

## Writing concurrent programs

* primitives, combinations, abstractions
    * primitives: channels
    * combinations: `put`, `get`, `select`
    * abstractions: "megachannels", something in, something out

* creating channels
    

* combining for dataflow
    

* Timing, backpressure, reactive programming

## Parallelism, non-concurrent programs

* The various forms of pipe

* How to get data in and out, from non-concurrent programs